# Capstone Project 
## Part 2 - Identifying 'systemised' listings

## Network Analysis - Data Merging

In [1]:
import pandas as pd
import numpy as np
import psycopg2
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import networkx as nx

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

sns.set_theme(context='notebook', style='darkgrid', palette='deep', font_scale=1.5)

### Load data

In [2]:
# connect to database
db_user = 'postgres'
db_password = ''
db_host = 'localhost'
db_port = 5432
database = 'airbnb'

conn_str = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{database}'
conn = psycopg2.connect(conn_str)

### Oct-19 and Oct-18 data

#### Reviews data

In [38]:
# load data for Oct 19 reviews from database
df_reviews_19 = pd.read_sql('SELECT * FROM oct19_reviews', conn)
df_reviews_19.shape

(1453649, 6)

In [39]:
# load data for Oct 18 reviews from database
df_reviews_18 = pd.read_sql('SELECT * FROM oct18_reviews', conn)
df_reviews_18.shape

(1102255, 6)

In [41]:
df_reviews = pd.concat([df_reviews_19, df_reviews_18], axis=0)
df_reviews.drop_duplicates(subset=['id'], inplace=True)
df_reviews.shape

(1752751, 6)

In [42]:
# drop null comments
df_reviews.dropna(inplace=True)
df_reviews.shape

(1751022, 6)

In [43]:
# drop reviews containing 'This is an automated posting'
automated = df_reviews[df_reviews.comments.str.contains('This is an automated posting')].index
df_reviews.drop(index=automated, inplace=True)
df_reviews.shape

(1706982, 6)

#### Listings data

In [44]:
# load data for Oct 19 listings from database
df_listings_19 = pd.read_sql('SELECT * FROM oct19_listings', conn)
df_listings_19.shape

(83887, 106)

In [45]:
# load data for Oct 18 listings from database
df_listings_18 = pd.read_sql('SELECT * FROM oct18_listings', conn)
df_listings_18.shape

(74153, 96)

In [46]:
cols = ['id', 'host_id', 'host_name']
df_listings = pd.concat([df_listings_19[cols], df_listings_18[cols]], axis=0)
df_listings.drop_duplicates(subset=['id', 'host_id'], inplace=True)
df_listings.shape

(109217, 3)

### Merge data

In [47]:
# create subsets with relevant columns for merging
df1 = df_reviews[['listing_id', 'reviewer_id', 'reviewer_name', 'id', 'date', 'comments']].copy()
df2 = df_listings[['id', 'host_id', 'host_name']].copy()

In [48]:
# rename id as listing_id for merge
df2.rename(columns={'id': 'listing_id'}, inplace=True)

In [51]:
# merge dataframes
df = pd.merge(df1, df2, how='left', on='listing_id')
df.shape

(1707049, 8)

In [52]:
df.dropna(inplace=True)
df.drop_duplicates(subset=['id'], inplace=True)
df.shape

(1706547, 8)

In [53]:
df.reviewer_id = df.reviewer_id.astype(int)

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1706547 entries, 0 to 1707048
Data columns (total 8 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   listing_id     int64 
 1   reviewer_id    int64 
 2   reviewer_name  object
 3   id             int64 
 4   date           object
 5   comments       object
 6   host_id        int64 
 7   host_name      object
dtypes: int64(4), object(4)
memory usage: 117.2+ MB


In [55]:
# write merged data to CSV
df.to_csv('../capstone-data-airbnb/merged_network_data_oct19.csv')

### Dec-20 data

#### Reviews data

In [18]:
# load data for Dec 20 reviews from database
df_reviews_20 = pd.read_sql('SELECT * FROM dec20_reviews', conn)
df_reviews_20.shape

(1178933, 6)

In [19]:
# drop null comments
df_reviews_20.dropna(inplace=True)
df_reviews_20.shape

(1178411, 6)

In [20]:
# drop reviews containing 'This is an automated posting'
automated = df_reviews_20[df_reviews_20.comments.str.contains('This is an automated posting')].index
df_reviews_20.drop(index=automated, inplace=True)
df_reviews_20.shape

(1158062, 6)

#### Listings data

In [21]:
# load data for Dec 20 listings from database
df_listings_20 = pd.read_sql('SELECT * FROM dec20_listings', conn)
df_listings_20.shape

(77136, 74)

### Merge data

In [22]:
# create subsets with relevant columns for merging
df1 = df_reviews_20[['listing_id', 'reviewer_id', 'reviewer_name', 'id', 'date', 'comments']].copy()
df2 = df_listings_20[['id', 'host_id', 'host_name']].copy()

In [23]:
# rename id as listing_id for merge
df2.rename(columns={'id': 'listing_id'}, inplace=True)

In [24]:
# merge dataframes
df = pd.merge(df1, df2, how='left', on='listing_id')
df.shape

(1158062, 8)

In [25]:
df.dropna(inplace=True)
df.drop_duplicates(subset=['id'], inplace=True)
df.shape

(1156947, 8)

In [26]:
df.reviewer_id = df.reviewer_id.astype(int)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1156947 entries, 0 to 1158061
Data columns (total 8 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   listing_id     1156947 non-null  int64 
 1   reviewer_id    1156947 non-null  int64 
 2   reviewer_name  1156947 non-null  object
 3   id             1156947 non-null  int64 
 4   date           1156947 non-null  object
 5   comments       1156947 non-null  object
 6   host_id        1156947 non-null  int64 
 7   host_name      1156947 non-null  object
dtypes: int64(4), object(4)
memory usage: 79.4+ MB


In [51]:
df.comments = df.comments.map(lambda x: x.replace('\r', ''))

In [53]:
# write merged data to CSV
df.to_csv('../capstone-data-airbnb/merged_network_data_dec20.csv', index=False)